In [1]:
%load_ext autoreload
%autoreload 2
    
!pip install pandas_gbq
import numpy as np
import pandas as pd
import pandas_gbq as bq
import re
import json
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from google.oauth2 import service_account
pd.options.display.float_format = "{:.4f}".format
from mmm_eval.comparison.load import get_datasets

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [2]:
from mmm_eval.comparison.load import get_datasets
#stage = "warchest-staging-323804"  # "warchest-develop"
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/samuelmccormick/.config/gcloud/application_default_credentials.json"
project_id = "mtx-dataos-datalake-prod"
customer_id = "bank_australia_43"
dataset_name = customer_id + '_datamart'
customer_name = customer_id.rpartition("_")[0]
data_version = "2025-07-10T05:08:31.208146"
#datasets = get_datasets(project_id, dataset_name, data_version, node_filter="default.default.home_loans")

## Design

Pass the following args:

- Customer slug
- Data version
- Holiday names
- Pipeline link

then

- process all datasets and merge them together
- call functions to construct data configs for each framework
- transformations
  - PyMC: maxabs scale all control columns
  - Meridian: add dummy geo column

## Turn into DFs for PyMC
- Restrict to node(s) of interest
- Convert all datasets to weekly
- Load holidays and process them
- Merge together into a single dataframe
- Set up config fields based on dataset column mapping

In [3]:
# import mmm_eval.comparison.utils as utils
# from mmm_eval.comparison.load import aggregate_to_node_level, convert_df_to_weekly

# sales_weekly = convert_df_to_weekly(
#     datasets["sales_snapshot"],
#     numerical_columns=["quantity", "value"],
#     downsample_method_to_daily={"value": utils.DownsampleMethod.UNIFORM, "quantity": utils.DownsampleMethod.UNIFORM},
#     agg_method_to_weekly={"value": "sum", "quantity": "sum"},
# )

# sales = aggregate_to_node_level(
#     sales_weekly, extra_group_cols=[], agg_mapping={"quantity": "sum", "value": "sum"}
# )

# sales_proc = sales[["date", "quantity", "value"]].set_index("date").rename(columns={"value": "revenue"})
# sales_proc.head()

In [4]:
# drop_columns = [
#     "impressions",
#     "funnel_stage",
#     "tarp",
#     "fees",
# ]

# if "impressions" in datasets["paid_media_snapshot"].columns:
#     datasets['paid_media_snapshot'].drop(columns=drop_columns, inplace=True)

# # Convert to weekly then aggregate to node or lower level
# paid_media_weekly = convert_df_to_weekly(
#     datasets['paid_media_snapshot'],
#     numerical_columns=["spend"],
#     downsample_method_to_daily={
#         "spend": utils.DownsampleMethod.UNIFORM,
#     },
#     agg_method_to_weekly={
#         "spend": "sum",
#     },
# )

# paid_media = aggregate_to_node_level(
#     paid_media_weekly, extra_group_cols=["media_channel", "marketing_spend_impact"], agg_mapping={"spend": "sum"}
# )

In [5]:
# pivot out by spend impact
# pivoted = paid_media.pivot(columns=["media_channel", "marketing_spend_impact"], values="spend", index="date")
# pivoted.columns = ['_'.join(col).strip() for col in pivoted.columns.values]
# paid_media_pivoted = pivoted.fillna(0)
# paid_media_pivoted.head()

In [6]:
# pricing_weekly = convert_df_to_weekly(
#     datasets["pricing_snapshot"],
#     numerical_columns=["retail_price"],
#     downsample_method_to_daily={
#         "retail_price": utils.DownsampleMethod.REPLICA,
#     },
#     agg_method_to_weekly={
#         "retail_price": "mean",
#     },
# )

# pricing = aggregate_to_node_level(
#     pricing_weekly, extra_group_cols=["company"], agg_mapping={"retail_price": "mean"}
# )
# # alternative is to include competitor prices as separate cols
# pricing = pricing[pricing["company"]=="bank_australia"]
# pricing = pricing[["date", "retail_price"]].set_index("date").ffill()
# pricing.head()

In [7]:
# events_weekly = convert_df_to_weekly(
#     datasets["events_snapshot"],
#     numerical_columns=["duration",],
#     downsample_method_to_daily=utils.DownsampleMethod.REPLICA,
#     agg_method_to_weekly={
#         "duration": "sum",
#     },
# )

# events = aggregate_to_node_level(
#     events_weekly, extra_group_cols=["event_type", "event_purpose"], agg_mapping={"duration": "sum"}
# )
# events

In [8]:
# binary_matrix = pd.crosstab(events["date"], events["event_purpose"])
# binary_matrix = binary_matrix.astype(bool).astype(int)
# events_dense = binary_matrix.reindex(sales_proc.index, fill_value=0)
# events_dense.head()

In [9]:
# merged = pd.merge(sales_proc, paid_media_pivoted, left_index=True, right_index=True)
# merged = pd.merge(merged, pricing, left_index=True, right_index=True)
# merged = pd.merge(merged, events_dense, left_index=True, right_index=True)
# merged.head()

# Fetch relevant holidays

In [10]:
# from random pipeline
holidays = pd.read_parquet("gs://mtx-ths-uat-au-theseus-pipeline-artifacts/artifacts/816012989105/theseus-simply-energy-20250709103055/run-feature-store-2_-5522238876388687872/model_output/export/holidays.parquet")
holidays.head()

,period_start,variable,value
0,2009-12-28,ANZAC Day,0.0000
1,2010-01-04,ANZAC Day,0.0000
2,2010-01-11,ANZAC Day,0.0000
3,2010-01-18,ANZAC Day,0.0000
4,2010-01-25,ANZAC Day,0.0000


In [11]:
# model_holidays = ["Christmas Day", "Good Friday", "New Year's Day"]
# holidays = holidays[(holidays["variable"].isin(model_holidays))].rename(columns={"period_start": "date"})
# holidays = holidays.pivot(index="date", columns="variable", values="value")

# merged = pd.merge(merged, holidays, left_index=True, right_index=True)
# merged

In [29]:
from mmm_eval.comparison.process import load_and_process_datasets

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/samuelmccormick/.config/gcloud/application_default_credentials.json"
customer_id = "belong_64"
data_version = "2025-07-07T05:59:08.517992"


pipeline_path = "gs://mtx-ths-uat-au-theseus-pipeline-artifacts/artifacts/816012989105/theseus-belong-20250714150222/run-feature-store-2_490770313592700928/model_output/export"
holidays_whitelist = ["new_year's_day_aus",
  'christmas_day_aus',
  'good_friday_aus',
  'eofy_aus']
# why are some of these not showing up?
externals_whitelist = ['foot_traffic_vol_feat_weekly_diff',
  'target_cash_rate_smooth_jump',
  'target_cash_rate_rolling_mean',
  'consumer_sentiment_feat_centered',
  'cpi_trimmed_mean_feat_smoothed',
  'any_lockdown_2021',
  'migration_total_arrivals_rolling_scaled',
                      ]
node_filter = "default.default.internet"
datasets_processed = load_and_process_datasets(customer_id, data_version, pipeline_path,
                                               holidays_whitelist, externals_whitelist, node_filter=node_filter)

2025-07-15 14:39:22,584 - mmm_eval.comparison.process - INFO - Loading datasets for customer belong_64 with data version 2025-07-07T05:59:08.517992 and node filter default.default.internet
/Users/samuelmccormick/work/mmm-eval/.venv/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
2025-07-15 14:39:29,337 - mmm_eval.comparison.load - INFO - Loading brand_snapshot


ValueError: Cannot set a DataFrame with multiple columns to the single column node

In [13]:
#datasets_processed

In [14]:
from mmm_eval.comparison.dataset_processor import DatasetProcessor

dp = DatasetProcessor.from_raw_data(datasets_processed, customer_id, data_version, holidays_whitelist,
                                    pipeline_path, node_filter=node_filter)

2025-07-15 09:48:43,024 - mmm_eval.comparison.dataset_processor - INFO - Created DatasetProcessor from raw data with shape (313, 16)


## Construct PyMC dataset and config

In [15]:
# import os

# os.listdir("../../comparison")

['constants.py',
 'dataset_processor.py',
 '__init__.py',
 'bank_aus_hl.parquet',
 '__pycache__',
 'utils.py',
 'scripts',
 'process.py',
 'dataloader.py',
 '.ipynb_checkpoints',
 'load.py',
 'data',
 'notebooks']

In [16]:
datasets_processed.to_parquet("../../comparison/data/bank_aus_hl.parquet")

In [17]:
#pymc_dataset = dp.get_pymc_dataset()

In [18]:
from pymc_marketing.mmm import GeometricAdstock, LogisticSaturation
from mmm_eval.adapters.schemas import PyMCFitSchema, PyMCModelSchema
from mmm_eval import PyMCConfig, run_evaluation

col_map = dp.get_pymc_column_map()
#print(col_map.keys())

fast_fit_config = PyMCFitSchema(draws=1000, tune=1000, chains=4, target_accept=0.95, random_seed=42)
model_config = PyMCModelSchema(date_column="date", channel_columns=col_map["channel_columns"],
                               control_columns=col_map["control_columns"],
                               adstock=GeometricAdstock(l_max=8), saturation=LogisticSaturation(),
                               time_varying_intercept=False, time_varying_media=False,
                               yearly_seasonality=2)
config = PyMCConfig(pymc_model_config=model_config, fit_config=fast_fit_config, response_column="quantity",
                    revenue_column="revenue")

### Run evaluation doesn't work - trying to debug why

In [19]:
# import pymc_marketing
# pymc_marketing.__file__

In [20]:
# import importlib

# # Reload the deepest dependency first
# import pymc_marketing.model_builder
# importlib.reload(pymc_marketing.model_builder)

# # Reload the module that uses model_builder
# import pymc_marketing.mmm
# importlib.reload(pymc_marketing.mmm)

# # Reload your adapter module
# import mmm_eval.adapters.pymc
# importlib.reload(mmm_eval.adapters.pymc)

# # Now re-import PyMCAdapter from the reloaded module
# from mmm_eval.adapters.pymc import PyMCAdapter

# # Recreate adapter
# adapter = PyMCAdapter(config)

# adapter.fit(pymc_dataset.rename(columns={"quantity": "response"}))
# adapter.predict(pymc_dataset.rename(columns={"quantity": "response"}))

In [21]:
#pymc_dataset.columns

                                                               
#result = run_evaluation(framework="pymc_marketing", config=config, data=pymc_dataset)

## Construct Meridian dataset and config

In [22]:
col_map = dp.get_meridian_column_map()
col_map

{'media_channels': ['Meta_product',
  'Display_brand',
  'Video_brand',
  'OOH_brand',
  'Display_product',
  'Search_product'],
 'control_columns': ['discount interest rate_event',
  'Good Friday',
  "New Year's Day",
  'Christmas Day',
  'government scheme launch_event',
  'consumer_sentiment_feat_centered',
  'cpi_raw_trimmed_mean_quad_interpolated'],
 'non_media_treatment_columns': ['retail_price']}

In [23]:
from mmm_eval.adapters.schemas import MeridianModelSpecSchema, MeridianSamplePosteriorSchema, MeridianInputDataBuilderSchema
from meridian import constants
import tensorflow_probability as tfp
from meridian.model import prior_distribution

roi_mu = 0.2     # Mu for ROI prior for each media channel.
roi_sigma = 0.9  # Sigma for ROI prior for each media channel.
prior = prior_distribution.PriorDistribution(
    roi_m=tfp.distributions.LogNormal(roi_mu, roi_sigma, name=constants.ROI_M)
)

# set up Meridian configs
model_spec_config = MeridianModelSpecSchema(prior=prior)
sample_posterior_config = MeridianSamplePosteriorSchema(n_chains=4, n_adapt=200, n_burnin=200, n_keep=400)
idb_config = MeridianInputDataBuilderSchema(date_column="date", media_channels=col_map["media_channels"],
                                            channel_spend_columns=col_map["media_channels"],
                                            non_media_treatment_columns=col_map.get("non_media_treatment_columns"),
                                            control_columns=col_map.get("control_columns"),
                                            response_column="quantity")

In [24]:
from mmm_eval.configs.configs import MeridianConfig

meridian_config = MeridianConfig(input_data_builder_config=idb_config, model_spec_config=model_spec_config,
                                 sample_posterior_config=sample_posterior_config, response_column="quantity",
                                 revenue_column="revenue")

In [25]:
meridian_dataset = dp.get_meridian_dataset()

2025-07-15 09:48:43,474 - mmm_eval.comparison.dataset_processor - INFO - Transformed dataset for Meridian with shape (313, 17)


In [26]:
result = run_evaluation(framework="meridian", config=meridian_config, data=meridian_dataset,
                        test_names=["accuracy"])

2025-07-15 09:48:43,745 - mmm_eval.core.validation_test_orchestrator - INFO - Running test: accuracy
2025-07-15 09:48:43,746 - mmm_eval.core.base_validation_test - INFO - Splitting data into train and test sets for accuracy test
/Users/samuelmccormick/work/mmm-eval/.venv/lib/python3.11/site-packages/meridian/model/model.py:66: UserWarning: In a nationally aggregated model, the `media_effects_dist` will be reset to `normal`.
  warnings.warn(
I0000 00:00:1752536924.108944  424723 service.cc:148] XLA service 0x600002164000 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1752536924.108972  424723 service.cc:156]   StreamExecutor device (0): Host, Default Version
I0000 00:00:1752536924.115455  424723 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
/Users/samuelmccormick/work/mmm-eval/.venv/lib/python3.11/site-packages/meridian/model/prior_distribution.py:912: UserWarning

In [27]:
result

,general_metric_name,specific_metric_name,metric_value,metric_pass,test_name,timestamp
0,mape,mape,0.2066,False,accuracy,2025-07-15T09:50:13.807964
1,r_squared,r_squared,0.0267,False,accuracy,2025-07-15T09:50:13.807964


In [28]:
#meridian_dataset

## Hypothesis: Meridian is somehow using revenue as the response column